In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
final_df = pd.read_csv("C:\\Users\\alama\\Downloads\\portfolio project\\portfolio project\\Dataset\\Final.csv")

In [3]:
final_df.head()

,teacher_number_of_previously_posted_projects,project_is_approved,quantity,price,project_title_art,project_title_book,project_title_books,project_title_building,project_title_class,project_title_classroom,...,"project_subject_categories_Music & The Arts, Applied Learning","project_subject_categories_Music & The Arts, Health & Sports","project_subject_categories_Music & The Arts, History & Civics","project_subject_categories_Music & The Arts, Special Needs","project_subject_categories_Music & The Arts, Warmth, Care & Hunger",project_subject_categories_Special Needs,"project_subject_categories_Special Needs, Health & Sports","project_subject_categories_Special Needs, Music & The Arts","project_subject_categories_Special Needs, Warmth, Care & Hunger","project_subject_categories_Warmth, Care & Hunger"
0,0,0,3,14.59,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,3,14.59,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,3,14.98,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,3,14.98,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,3,14.59,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
final_df.shape

(650007, 310)

In [5]:
X = final_df.drop(columns='project_is_approved',axis=1)
y = final_df['project_is_approved']

In [6]:
import os
import pickle
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Step 1: Ensure the output directory exists
output_dir = "C:\\Users\\alama\\Downloads\\portfolio project\\portfolio project\\pickle_data"
os.makedirs(output_dir, exist_ok=True)

# Step 2: Load and split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Apply SMOTE (handle memory issues)
try:
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
except MemoryError:
    print("MemoryError: Reduce the dataset size or use a smaller subset of features.")
    raise

# Step 4: Scale the features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# Save the StandardScaler
scaler_path = os.path.join(output_dir, "scaler.pkl")
with open(scaler_path, "wb") as scaler_file:
    pickle.dump(scaler, scaler_file)

# Step 5: Apply PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Retain 95% variance
X_resampled_pca = pca.fit_transform(X_resampled_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Save the PCA model
pca_path = os.path.join(output_dir, "pca.pkl")
with open(pca_path, "wb") as pca_file:
    pickle.dump(pca, pca_file)

print(f"StandardScaler and PCA models have been saved in {output_dir}.")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [7]:
X_train.shape, X_test.shape

((520005, 309), (130002, 309))

In [8]:
X_resampled_pca.shape, X_test_pca.shape

((826058, 272), (130002, 272))

In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import time

def evaluate_models(X_train, X_test, y_train, y_test):
    """
    Fits multiple classifiers and evaluates their performance on the training and test datasets.

    Args:
        X_train (array-like): Training feature set.
        X_test (array-like): Test feature set.
        y_train (array-like): Training target set.
        y_test (array-like): Test target set.

    Returns:
        pd.DataFrame: DataFrame containing model names and their accuracies on training and test sets.
    """
    # Initialize models with optimized hyperparameters
    models = {
        "Random Forest": RandomForestClassifier(
            n_estimators=100, max_depth=10, n_jobs=-1, random_state=42
        ),
        "XGBoost": XGBClassifier(
            n_estimators=100, max_depth=10, n_jobs=-1, 
            use_label_encoder=False, eval_metric='logloss', random_state=42
        ),
        "Linear SVC": LinearSVC(random_state=42, max_iter=1000),
        "Naive Bayes": GaussianNB()
    }
    
    # List to store evaluation results
    results = []

    # Loop through each model, fit, and evaluate
    for name, model in models.items():
        print(f"Training {name}...")
        start_time = time.time()
        
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predict on training and test sets
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        # Calculate accuracies
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        
        # Append results
        results.append({
            "Algorithm": name, 
            "Train Accuracy": train_accuracy, 
            "Test Accuracy": test_accuracy,
            "Training Time (s)": time.time() - start_time
        })
        
        print(f"{name} completed in {time.time() - start_time:.2f} seconds.")
    
    # Create a DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

# Example usage:
# Assuming X_train, X_test, y_train, y_test are preprocessed and ready
# results_df = evaluate_models(X_train, X_test, y_train, y_test)
# print(results_df)

In [10]:
results_df = evaluate_models(X_resampled_pca, X_test_pca, y_resampled, y_test)

Training Random Forest...
Random Forest completed in 90.01 seconds.
Training XGBoost...


c:\Users\alama\anaconda3\envs\py39_env\lib\site-packages\xgboost\core.py:158: UserWarning: [10:19:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost completed in 28.03 seconds.
Training Linear SVC...
Linear SVC completed in 26.44 seconds.
Training Naive Bayes...
Naive Bayes completed in 5.76 seconds.


In [11]:
results_df

,Algorithm,Train Accuracy,Test Accuracy,Training Time (s)
0,Random Forest,0.839286,0.782980,90.010508
1,XGBoost,0.999282,0.985700,28.029335
2,Linear SVC,0.671580,0.655144,26.441734
3,Naive Bayes,0.602989,0.464131,5.759893
